In [1]:
import tensorflow as tf

print("Physical GPUs:", tf.config.list_physical_devices("GPU"))
print("CUDA built:", tf.test.is_built_with_cuda())

Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
CUDA built: True


2025-05-01 00:03:34.566606: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-05-01 00:03:34.676619: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-01 00:03:34.676660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.785GHz coreCount: 68 deviceMemorySize: 10.00GiB deviceMemoryBandwidth: 707.88GiB/s
2025-05-01 00:03:34.676967: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2025-05-01 00:03:34.679925: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-05-01 00:03:34.681520: I tensorflow/stream_execut

In [2]:
import sys
sys.path.insert(0, 'seizure-detection-neonates')
from models import copy_model, model_DL2
from utils import rearrange_epochs
import numpy as np
import scipy.io as sio
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [3]:
# Step 1: Create a model and load weights from pre-training on Duke dataset. Adjust path in WEIGHTS_FILE variable
X = np.empty([100,12,256]) #batch size, channel number, samples number
Y = np.empty([100,1])
model=model_DL2(X,Y)
WEIGHTS_FILE='weights/DL2_Duke_pretrained.model_weights'
model.load_weights(WEIGHTS_FILE)

2025-05-01 00:03:40.454573: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-05-01 00:03:40.470037: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3792915000 Hz
2025-05-01 00:03:40.475011: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9908000b70 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-05-01 00:03:40.475043: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-05-01 00:03:40.654180: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-01 00:03:40.654303: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x263b5650 initialized for platform CUDA (this does not guarantee that XL

In [4]:
# Step 2: Create a model suited for Helsinki dataset and copy weights to it
X_helsinki = np.empty([100,18,256])
Y_helsinki = np.empty([100,1])
model_helsinki = model_DL2(X_helsinki,Y_helsinki)
model_helsinki = copy_model(model,model_helsinki)

In [5]:
# Step 3: Load data of one subject from Helsinki dataset. 
# Extraction is done via adjusted code from Tapani et al. repository
# Seizure annotations are by consensus of 3 raters
matdata_fname = 'preprocessed_feats/helsinki_eeg1_feats.mat'
annot_fname = 'preprocessed_feats/helsinki_eeg1_cons_chan_annot.mat'
matdata = sio.loadmat(matdata_fname)
matannot = sio.loadmat(annot_fname)

In [9]:
# Step 4: Transform matdata and matannot into a suitable structure for prediction, and predict
# Duke dataset is trained on different scale of the data, so we need to add scaling coefficient sc

In [6]:
epochs, annot, _ = rearrange_epochs(matdata['all_feats'], matannot['annot'], 1,
                                                      None, n_chan=18)
sc = 0.001
epochs = epochs*sc
pred = model_helsinki.predict(epochs, verbose = 1)

NaN epochs: 225
correct epochs: 1522


2025-05-01 00:08:24.096238: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2025-05-01 00:10:11.999487: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2025-05-01 00:14:24.194147: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2025-05-01 00:14:24.282554: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1522/1522 [==============================] - 389s 256ms/step


In [7]:
# Simple AUC score with annotations and predictions. Post-processing usually improves this AUC value.
roc_auc_score(annot,pred)

0.5378416518122402